### Generic Imports

In [ ]:
from gladier import GladierBaseTool, GladierBaseClient, generate_flow_definition
from pprint import pprint

### Function that performs the encrypt operation

In [ ]:
def encrypt(**data):
    import os 
    from cryptography.fernet import Fernet
    from cryptography.hazmat.primitives import hashes
    from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
    from cryptography.hazmat.primitives.kdf.concatkdf import ConcatKDFHash
    import base64

    password = bytes(data['key'], 'utf-8')
    # salt= b'\00\00\00\00\00\00\00\00\00\00\00\00\00\00\00\00'
    # salt = os.urandom(16)
    ckdf = ConcatKDFHash(algorithm=hashes.SHA256(),length=32,otherinfo=None,)
    #kdf = PBKDF2HMAC(algorithm=hashes.SHA256(),length=32,salt=salt,iterations=100000)
    key = base64.urlsafe_b64encode(ckdf.derive(password))
    fernet = Fernet(key)
    
    infile= data['encrypt_input']
    

    if '~' in infile:
        infile = os.path.expanduser(infile)
    
    outfile= infile+".aes"
    
    if os.path.isdir(infile):
        raise Exception("Please input the path to a file or a tarred directory.")

    # opening the original file to encrypt
    with open(infile, 'rb') as file:
        original = file.read()
        
    # encrypting the file
    encrypted = fernet.encrypt(original)
    
    # opening the file in write mode and 
    # writing the encrypted data
    with open(outfile, 'wb+') as encrypted_file:
        encrypted_file.write(encrypted)

    return key

### Creating a GladierBaseTool class for the tool

In [ ]:
@generate_flow_definition
class Encrypt(GladierBaseTool):
    funcx_functions = [encrypt]
    required_input = [
        'encrypt_input',
        'key', 
        'funcx_endpoint_compute'
        ]

### Creating a Gladier Client class to run the flow

In [ ]:
@generate_flow_definition
class EncryptGladier(GladierBaseClient):
    gladier_tools = [Encrypt]

### Running the flow

Enter the appropriate values for ```encrypt_input```, ```key```, and ```funcx_endpoint_compute```

In [ ]:
hello_cli = EncryptGladier()
flow_input = {
'input': {
    #encrypt_input is the path to the file, key is the password (string), and funcx_endpoint_compute is the UUID of the func endpoint
    'encrypt_input': ,
    'key':'rohith',
    'funcx_endpoint_compute': 'a43e4540-98ba-4784-ade1-357f0eb4ddc8',
    }
}

# print(encrypt(flow_input))
flow = hello_cli.run_flow(flow_input=flow_input)

# Watch the progress, get the result.
hello_cli.progress(flow['action_id'])
pprint(hello_cli.get_status(flow['action_id']))